# Курсовая работа студента факультета искусственного интеллекта онлайн-университета [Geek University](https://geekbrains.ru/geek_university)

## В рамках курса "Python для Data Science"  

Полная версия

**Автор**: Кабанов Сергей  
**Преподаватель**: Ширкин Сергей

**Материалы к проекту (файлы)**:  
train.csv  
test.csv

**Задание**:  
Используя данные из train.csv, построить модель для предсказания цен на недвижимость (квартиры). С помощью полученной модели предсказать цены для квартир из файла test.csv.

**Целевая переменная**:  
Price

**Основная метрика**:  
R2 - коэффициент детерминации (sklearn.metrics.r2_score)

**Вспомогательная метрика**:  
MSE - средняя квадратичная ошибка (sklearn.metrics.mean_squared_error)

In [1]:
RANDOM_STATE = 42

### Data load (загрузка данных)

В данном разделе выполним загрузку данных в объекты типа DataFrame и проверим корректность данных.

In [2]:
import pandas as pd

TRAIN_DATA_FILE = 'train.csv'
TEST_DATA_FILE = 'test.csv'

In [3]:
# загрузка данных
data_train = pd.read_csv(TRAIN_DATA_FILE)
data_test = pd.read_csv(TEST_DATA_FILE)

Необходимо проверить загруженные данные.

In [4]:
# проверяем количество объектов и признаков
data_train.shape, data_test.shape

((10000, 20), (5000, 19))

In [5]:
# смотрим как выглядят данные, правильно ли прочитался файл, нет ли проблем с разделителями/кодировкой
data_train.head()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price
0,14038,35,2.0,47.981561,29.442751,6.0,7,9.0,1969,0.089040,B,B,33,7976,5,NaN,0,11,B,184966.930730
1,15053,41,3.0,65.683640,40.049543,8.0,7,9.0,1978,0.000070,B,B,46,10309,1,240.0,1,16,B,300009.450063
2,4765,53,2.0,44.947953,29.197612,0.0,8,12.0,1968,0.049637,B,B,34,7759,0,229.0,1,3,B,220925.908524
3,5809,58,2.0,53.352981,52.731512,9.0,8,17.0,1977,0.437885,B,B,23,5735,3,1084.0,0,5,B,175616.227217
4,10783,99,1.0,39.649192,23.776169,7.0,11,12.0,1976,0.012339,B,B,35,5776,1,2078.0,2,4,B,150226.531644


In [6]:
# аналогично для test
data_test.head()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2
0,725,58,2.0,49.882643,33.432782,6.0,6,14.0,1972,0.310199,B,B,11,2748,1,NaN,0,0,B
1,15856,74,2.0,69.263183,NaN,1.0,6,1.0,1977,0.075779,B,B,6,1437,3,NaN,0,2,B
2,5480,190,1.0,13.597819,15.948246,12.0,2,5.0,1909,0.000000,B,B,30,7538,87,4702.0,5,5,B
3,15664,47,2.0,73.046609,51.940842,9.0,22,22.0,2007,0.101872,B,B,23,4583,3,NaN,3,3,B
4,14275,27,1.0,47.527111,43.387569,1.0,17,17.0,2017,0.072158,B,B,2,629,1,NaN,0,0,A


Имеем 10000 квартир в наборе train, и 5000 квартир для которых нужно сделать предсказание. Данные загружены корректно.

### Data analysis (анализ данных)

В данном разделе выполним первичный анализ данных на предмет аномалий, неадекватных/пропущенных значений. Попробуем понять что собой представляют признаки. Так же исправим не системные ошибки.

In [7]:
# Посмотрим на названия признаков, наличие пропущенных и нечисловых значений.
data_train.info();

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 20 columns):
Id               10000 non-null int64
DistrictId       10000 non-null int64
Rooms            10000 non-null float64
Square           10000 non-null float64
LifeSquare       7887 non-null float64
KitchenSquare    10000 non-null float64
Floor            10000 non-null int64
HouseFloor       10000 non-null float64
HouseYear        10000 non-null int64
Ecology_1        10000 non-null float64
Ecology_2        10000 non-null object
Ecology_3        10000 non-null object
Social_1         10000 non-null int64
Social_2         10000 non-null int64
Social_3         10000 non-null int64
Healthcare_1     5202 non-null float64
Helthcare_2      10000 non-null int64
Shops_1          10000 non-null int64
Shops_2          10000 non-null object
Price            10000 non-null float64
dtypes: float64(8), int64(9), object(3)
memory usage: 1.5+ MB


Ряд признаков имеет вполне понятный смысл (от Id до HouseYear, а так же Price). Остальные признаки вероятно говорят о характеристике района, чем самой квартиры. Экологическая ситуация (загрязнение воздуха, город/загород?), социальные показатели (школы, сады, криминал?), показатели здравоохранения (характеристики больниц? аптеки?), а так же информация по магазинам (удаленность, количество, размеры?)

Всего у двух признаков имеются пропущенные значения. Предварительно данные для жилой площади можно попробовать заполнить статистическим значениями. А вот отсутсвие данных по признаку Healthcare_1 может быть вызвано тем, что для этих квартир данная оценка например равна 0, что является нормальным значением.

Три признака являются строками. Их нужно будет приводить к числам.

In [8]:
# Посмотрим на статистики по признакам. Транспонируем вывод для удобства чтения.
data_train.describe().T

,count,mean,std,min,25%,50%,75%,max
Id,10000.0,8383.407700,4859.019020,0.000000,4169.500000,8394.500000,12592.500000,1.679800e+04
DistrictId,10000.0,50.400800,43.587592,0.000000,20.000000,36.000000,75.000000,2.090000e+02
Rooms,10000.0,1.890500,0.839512,0.000000,1.000000,2.000000,2.000000,1.900000e+01
Square,10000.0,56.315775,21.058732,1.136859,41.774881,52.513310,65.900625,6.410652e+02
LifeSquare,7887.0,37.199645,86.241209,0.370619,22.769832,32.781260,45.128803,7.480592e+03
KitchenSquare,10000.0,6.273300,28.560917,0.000000,1.000000,6.000000,9.000000,2.014000e+03
Floor,10000.0,8.526700,5.241148,1.000000,4.000000,7.000000,12.000000,4.200000e+01
HouseFloor,10000.0,12.609400,6.775974,0.000000,9.000000,13.000000,17.000000,1.170000e+02
HouseYear,10000.0,3990.166300,200500.261427,1910.000000,1974.000000,1977.000000,2001.000000,2.005201e+07
Ecology_1,10000.0,0.118858,0.119025,0.000000,0.017647,0.075424,0.195781,5.218671e-01


Пройдемся по признакам c понятным смыслом. Запишем вопросы и заметки.

Rooms:  
квартиры без комнат, возможно студии?  
квартира с 19 комнатами  

Square:  
квартиры с очень маленькой площадью, квартиры с площадью менее 15 метров выглядят подозрительно  
квартиры с очень большой площадью (разница между 3 квартилем и максимальным значением слишком большая)  

LifeSquare:  
аналогично Square  
пропущенные значения  

KitchenSquare:  
есть квартиры без кухни?  
максимальное значение похоже на год, вероятно опечатка при заполнении данных  

Floor:  
42 этаж выглядит подозрительно, учитывая, что третий квартиль равен 12  

HouseFloor:  
дома с нулем этажей?  
максимум равен 117, вероятно это опечатка и значение должно быть равно 17  

HouseYear:  
максимум выглядит как опечатка, 2.005201e+07 это скорее всего либо 2005, либо 201* какой-то год  


#### HouseYear

In [9]:
# посмотрим на слишком большие знаечения поля HouseYear
data_train.loc[data_train['HouseYear'] > 2020]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price
1497,10814,109,1.0,37.265070,20.239714,9.0,9,12.0,20052011,0.136330,B,B,30,6141,10,262.0,3,6,B,254084.534396
4189,11607,147,2.0,44.791836,28.360393,5.0,4,9.0,4968,0.319809,B,B,25,4756,16,2857.0,5,8,B,243028.603096


In [10]:
# заменим 20052011 на 2005
data_train.loc[data_train['HouseYear'] == 20052011, 'HouseYear'] = 2005

# заменим 4968 на 1968
data_train.loc[data_train['HouseYear'] == 4968, 'HouseYear'] = 1968

#### HouseFloor

In [13]:
# посмотрим на большие значения поля HouseFloor
data_train.loc[data_train['HouseFloor'] > 50]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price
6131,10806,5,1.0,51.944587,48.709601,1.0,6,99.0,1977,0.150818,B,B,16,3433,4,2643.0,4,5,B,296127.115515
8599,9300,74,2.0,71.747869,74.579809,9.0,5,99.0,1977,0.075779,B,B,6,1437,3,NaN,0,2,B,243329.912579
8854,78,30,2.0,65.773749,66.811789,1.0,8,117.0,1977,0.000078,B,B,22,6398,141,1046.0,3,23,B,207248.370520


In [14]:
# заменим 99 на 9
data_train.loc[data_train['HouseFloor'] == 99, 'HouseFloor'] = 9

# заменим 117 на 17
data_train.loc[data_train['HouseFloor'] == 117, 'HouseFloor'] = 17

In [16]:
# посмотрим на квартиры с нулем этажей
data_train.loc[data_train['HouseFloor'] == 0]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price
7,11993,74,2.0,80.312926,NaN,0.0,14,0.0,1977,0.075779,B,B,6,1437,3,NaN,0,2,B,221244.156664
23,6641,54,3.0,118.907612,NaN,0.0,2,0.0,1977,0.006076,B,B,30,5285,0,645.0,6,6,B,571069.052600
26,4378,27,3.0,106.958871,0.641822,0.0,17,0.0,2018,0.072158,B,B,2,629,1,NaN,0,0,A,337299.867936
39,9371,23,2.0,60.503248,NaN,0.0,16,0.0,1977,0.034656,B,B,0,168,0,NaN,0,0,B,229778.057902
44,10521,38,3.0,104.211396,106.340403,0.0,20,0.0,2017,0.060753,B,B,15,2787,2,520.0,0,7,B,435462.048070
65,11398,62,1.0,41.194188,NaN,0.0,6,0.0,1977,0.072158,B,B,2,629,1,NaN,0,0,A,114063.092041
70,10362,27,2.0,79.227049,NaN,0.0,9,0.0,1977,0.072158,B,B,2,629,1,NaN,0,0,A,167023.973143
195,4645,54,2.0,76.312851,NaN,0.0,14,0.0,1977,0.006076,B,B,30,5285,0,645.0,6,6,B,466730.805333
223,4622,27,2.0,62.153994,36.094352,0.0,5,0.0,1977,0.072158,B,B,2,629,1,NaN,0,0,A,194091.992560
252,8139,27,2.0,58.553809,NaN,0.0,7,0.0,1977,0.072158,B,B,2,629,1,NaN,0,0,A,177511.314827


Квартир с признаком HouseFloor = 0 много (269 шт.), это явно системная аномалия. Так же похоже, что у большинства таких квартир признак HouseYear = 1977. Проверим это наблюдение.

In [17]:
# количество квартир с нулевым значением HouseFloor по году постройки дома
data_train.loc[data_train['HouseFloor'] == 0]['HouseYear'].value_counts()

1977    253
2016      6
2018      4
2015      4
2017      1
2014      1
Name: HouseYear, dtype: int64

Наблюдение подтвердилось. Нули по признаку HouseFloor нужно будет заполнять по статистическим данным.

#### Floor

In [18]:
# посмотрим на большие значения поля Floor
data_train.loc[data_train['Floor'] > 40]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price
2781,1674,76,2.0,74.344672,41.044132,5.0,42,48.0,2016,0.0,B,B,7,1660,39,1786.0,1,5,B,411691.504766


Вполне нормальное значение. Оставляем без изменений.

#### KitchenSquare

In [19]:
# посмотрим на большие значения поля KitchenSquare
data_train.loc[data_train['KitchenSquare'] > 20]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price
227,16395,2,3.0,79.722243,44.731219,72.0,12,16.0,1987,0.130618,B,B,39,10418,9,900.0,1,9,B,370148.625285
551,1315,6,1.0,48.128114,45.023531,48.0,21,1.0,2013,0.243205,B,B,5,1564,0,540.0,0,0,B,146950.912410
622,12924,30,3.0,92.473796,53.649526,22.0,3,2.0,2018,0.000078,B,B,22,6398,141,1046.0,3,23,B,165521.195968
1064,14656,62,1.0,47.100719,46.447960,2014.0,4,1.0,2014,0.072158,B,B,2,629,1,NaN,0,0,A,108337.484207
1077,9690,27,1.0,40.081042,37.834964,37.0,18,19.0,2019,0.211401,B,B,9,1892,0,NaN,0,1,B,93218.650461
1369,2371,27,2.0,68.841073,64.234956,66.0,4,2.0,2014,0.017647,B,B,2,469,0,NaN,0,0,B,189244.249909
1455,12507,54,2.0,79.810535,79.578961,78.0,10,15.0,2014,0.006076,B,B,30,5285,0,645.0,6,6,B,438708.707579
1804,11459,79,1.0,98.727990,49.781722,22.0,18,24.0,2019,0.050756,B,B,24,5469,1,145.0,0,1,B,351021.797311
1860,4265,161,2.0,53.216778,32.644859,53.0,7,17.0,1994,0.000699,B,B,14,3369,24,4129.0,0,3,B,261125.669724
1867,3267,58,3.0,138.980817,138.004600,43.0,2,2.0,1977,0.437885,B,B,23,5735,3,1084.0,0,5,B,169528.896664


Наблюдаем 48 не вполне адекватных значений. Где-то указан год, где-то площад кухни равна общей площади или жилой. Но есть и вполне нормальные значения (с учетом общей площади). Нужно будет считать отношение площади кухни к общей площади и править неадекватные значения. Сейчас оставляем без изменений.

Так же видим проблему с полями Floor и HouseFloor. Есть квартиры у которых Floor > HouseFloor.

Так же видим что у некоторых квартир не адекватное соотношение жилой площади к общей.

In [19]:
# посмотрим на маленькие значения поля KitchenSquare
data_train.loc[data_train['KitchenSquare'] < 1]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price
2,4765,53,2.0,44.947953,29.197612,0.0,8,12.0,1968,0.049637,B,B,34,7759,0,229.0,1,3,B,220925.908524
7,11993,74,2.0,80.312926,NaN,0.0,14,0.0,1977,0.075779,B,B,6,1437,3,NaN,0,2,B,221244.156664
23,6641,54,3.0,118.907612,NaN,0.0,2,0.0,1977,0.006076,B,B,30,5285,0,645.0,6,6,B,571069.052600
26,4378,27,3.0,106.958871,0.641822,0.0,17,0.0,2018,0.072158,B,B,2,629,1,NaN,0,0,A,337299.867936
39,9371,23,2.0,60.503248,NaN,0.0,16,0.0,1977,0.034656,B,B,0,168,0,NaN,0,0,B,229778.057902
42,9833,1,2.0,56.494318,54.723569,0.0,9,17.0,1977,0.007122,B,B,1,264,0,NaN,0,1,B,196078.907289
44,10521,38,3.0,104.211396,106.340403,0.0,20,0.0,2017,0.060753,B,B,15,2787,2,520.0,0,7,B,435462.048070
52,2301,1,2.0,61.400054,65.224603,0.0,17,22.0,2016,0.007122,B,B,1,264,0,NaN,0,1,B,199215.452229
65,11398,62,1.0,41.194188,NaN,0.0,6,0.0,1977,0.072158,B,B,2,629,1,NaN,0,0,A,114063.092041
70,10362,27,2.0,79.227049,NaN,0.0,9,0.0,1977,0.072158,B,B,2,629,1,NaN,0,0,A,167023.973143


У 697 квартир площадь кухни равна 0. Возможно это квартиры студии или данных по площади кузхни просто не было. Как вариант можно все неадекватные значения признака KitchenSquare занулить.

#### Floor > HouseFloor

In [20]:
# посмотрим на квартиры у которых этаж больше чем этажность дома, исключая дома с 0 этажей
data_train.loc[(data_train['Floor'] > data_train['HouseFloor']) & (data_train['HouseFloor'] > 0)]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price
14,10953,27,1.0,53.769097,52.408027,1.0,5,4.0,1977,7.215758e-02,B,B,2,629,1,NaN,0,0,A,140647.565937
16,2119,27,2.0,49.360648,31.993964,5.0,6,5.0,1983,5.181543e-02,B,B,5,1227,0,NaN,0,0,B,117000.381287
21,11935,27,2.0,64.711835,NaN,1.0,15,1.0,1977,2.114012e-01,B,B,9,1892,0,NaN,0,1,B,127200.026511
35,6486,200,3.0,85.280389,58.447967,9.0,6,5.0,1960,0.000000e+00,B,B,33,7425,1,NaN,2,5,B,402871.916317
51,10103,94,1.0,35.280894,23.354176,6.0,11,9.0,1971,2.827977e-01,B,B,33,8667,2,NaN,0,6,B,148862.210174
67,8443,57,2.0,45.964890,29.141212,5.0,7,5.0,1963,1.332153e-01,B,B,49,11395,3,1406.0,3,4,A,180538.898922
68,1063,77,1.0,42.678844,23.918634,8.0,14,12.0,1983,6.998930e-05,B,B,46,10309,1,240.0,1,16,B,179784.473334
72,14675,74,1.0,52.457589,52.423345,0.0,18,17.0,1977,7.577876e-02,B,B,6,1437,3,NaN,0,2,B,160292.163702
76,14020,34,1.0,47.909327,42.782385,1.0,2,1.0,2016,6.975336e-02,B,B,53,13670,4,NaN,1,11,B,181167.642404
86,9613,148,3.0,127.481583,NaN,1.0,16,1.0,2012,3.139246e-01,B,B,22,4625,11,3855.0,2,10,B,510700.444735


Таких квартир 1556 штук. В данных по этажу и этажности дома много ошибок. Скорее всего лучше будет сделать отдельный признак говорящий о том последний этаж у квартиры или первый, для тех квартир где это можно расчитать.

#### LifeSquare и Square

In [21]:
# посмотрим на квартиры с очень большой площадью, оставим только понятные столбцы
data_train.loc[data_train['Square'] > 200, ['Id', 'Rooms', 'Square', 'LifeSquare', 'KitchenSquare',
                                            'Floor', 'HouseFloor', 'Price']]

,Id,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,Price
1981,7917,0.0,212.932361,211.231125,0.0,2,3.0,302211.260887
1982,5548,5.0,275.645284,233.949309,26.0,12,37.0,455264.882666
4262,28,2.0,604.705972,NaN,1.0,17,18.0,187717.242538
4690,2307,1.0,409.425181,410.639749,10.0,4,4.0,90470.430830
6977,11602,2.0,641.065193,638.163193,10.0,20,19.0,133529.681562
9910,16568,4.0,200.334539,201.627361,25.0,1,2.0,528560.506016


Похоже, что у квартиры с Id 28, 2307, 11602 судя по их стоимости ошибка в полях LifeSquare и Square. Значения завышены в 10 раз. Для остальных квартир площадь выглядит нормально.

In [22]:
# Похоже, что у квартиры с Id 28, 2307, 11602 судя по их стоимости ошибка в полях LifeSquare и Square. 
# Значения завышены в 10 раз. Для остальных квартир площадь выглядит нормально.
# уменьшим площади в 10 раз для данных квартир
data_train.loc[data_train['Id'].isin([28, 2307, 11602]), ['Square', 'LifeSquare']] = \
                                        data_train.loc[data_train['Id'].isin([28, 2307, 11602]), ['Square', 'LifeSquare']] / 10

In [23]:
# проверим квартиры у которых LifeSquare > Square
data_train.loc[data_train['LifeSquare'] > data_train['Square']]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price
28,8054,23,1.0,42.530043,43.967759,1.0,3,9.0,2014,0.034656,B,B,0,168,0,NaN,0,0,B,95338.198549
44,10521,38,3.0,104.211396,106.340403,0.0,20,0.0,2017,0.060753,B,B,15,2787,2,520.0,0,7,B,435462.048070
52,2301,1,2.0,61.400054,65.224603,0.0,17,22.0,2016,0.007122,B,B,1,264,0,NaN,0,1,B,199215.452229
123,8753,25,3.0,85.952306,89.803753,1.0,4,3.0,2017,0.069753,B,B,53,13670,4,NaN,1,11,B,309688.592681
153,9870,62,1.0,51.831473,53.491301,1.0,5,1.0,2015,0.072158,B,B,2,629,1,NaN,0,0,A,131797.472284
178,2416,57,1.0,29.298168,29.770784,5.0,7,5.0,1964,0.133215,B,B,49,11395,3,1406.0,3,4,A,148991.265200
184,3398,74,1.0,48.743665,51.011232,1.0,12,20.0,2017,0.075779,B,B,6,1437,3,NaN,0,2,B,165071.825722
212,1748,88,2.0,5.497061,67.628717,1.0,24,22.0,1977,0.127376,B,B,43,8429,3,NaN,3,9,B,412511.088764
217,5540,27,3.0,57.643613,59.425078,9.0,16,10.0,1977,0.072158,B,B,2,629,1,NaN,0,0,A,198351.892455
234,12633,27,3.0,81.867166,81.884548,0.0,12,19.0,1977,0.211401,B,B,9,1892,0,NaN,0,1,B,182228.520030


Таких квартир 482 штуки. Разница между значениями небольшая. Можно будет взять максимальное значение для нового признака площади квартиры. Либо вообще не использовать LifeSquare, т.к. в нем много пропущенных значений.

In [24]:
# посмотрим на маленькие знаечния Square
data_train.loc[data_train['Square'] < 15, ['Id', 'Rooms', 'Square', 'LifeSquare', 'KitchenSquare',
                                            'Floor', 'HouseFloor', 'Price']]

,Id,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,Price
212,1748,2.0,5.497061,67.628717,1.0,24,22.0,412511.088764
1316,11526,1.0,4.633498,1.969969,1.0,18,1.0,107604.269441
1608,10202,1.0,2.596351,4.604943,1.0,3,25.0,137597.601458
3280,10527,1.0,4.380726,40.805837,1.0,10,17.0,97560.720383
3413,9487,1.0,5.129222,5.549458,1.0,1,1.0,369472.403061
4739,12676,3.0,13.784865,15.988889,7.0,4,5.0,78388.806186
4853,3224,0.0,2.377248,0.873147,0.0,1,0.0,126596.941798
4900,4504,3.0,4.390331,5.610772,1.0,8,19.0,161379.067034
6392,14786,1.0,1.136859,4.525736,1.0,3,1.0,181434.825589
8030,13265,3.0,4.823679,79.767964,0.0,6,17.0,237716.681261


Площади таких квартиры нужно будет заменять на статистические данные.

#### Rooms

In [25]:
# посмотрим на квартиры с 0 комнат
data_train.loc[data_train['Rooms'] == 0]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price
1397,12638,27,0.0,138.427694,136.215499,0.0,4,3.0,2016,0.075424,B,B,11,3097,0,NaN,0,0,B,268394.744389
1981,7917,27,0.0,212.932361,211.231125,0.0,2,3.0,2008,0.211401,B,B,9,1892,0,NaN,0,1,B,302211.260887
2269,7317,27,0.0,41.790881,NaN,0.0,13,0.0,1977,0.211401,B,B,9,1892,0,NaN,0,1,B,98129.976788
3911,770,28,0.0,49.483501,NaN,0.0,16,0.0,2015,0.118537,B,B,30,6207,1,1183.0,1,0,B,217009.338463
4366,456,6,0.0,81.491446,NaN,0.0,4,0.0,1977,0.243205,B,B,5,1564,0,540.0,0,0,B,212864.799112
4853,3224,27,0.0,2.377248,0.873147,0.0,1,0.0,1977,0.017647,B,B,2,469,0,NaN,0,0,B,126596.941798
6149,3159,88,0.0,38.697117,19.345131,9.0,9,16.0,1982,0.127376,B,B,43,8429,3,NaN,3,9,B,158998.110646
8834,9443,27,0.0,87.762616,85.125471,0.0,5,15.0,1977,0.211401,B,B,9,1892,0,NaN,0,1,B,219281.918007


Либо это квартиры студии, либо здесь ошибка при заполнении. Такие квартиры стоит заполнить либо 1, либо по статистическим данным.

In [26]:
# посмотрим на квартиры с большим количеством комнат
data_train.loc[data_train['Rooms'] > 5]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price
377,5927,57,10.0,59.056975,36.223072,10.0,22,22.0,2002,0.090799,B,B,74,19083,2,NaN,5,15,B,317265.323792
1454,8491,1,19.0,42.006046,21.779288,7.0,17,17.0,2014,0.007122,B,B,1,264,0,NaN,0,1,B,78364.616704
2170,14003,99,6.0,59.414334,38.702244,6.0,7,9.0,1969,0.033494,B,B,66,10573,1,1322.0,3,8,B,229661.964416
8849,14865,9,10.0,60.871266,38.420681,10.0,3,2.0,1994,0.161532,B,B,25,5648,1,30.0,2,4,B,172329.270863


In [27]:
# посмотрим медианные значения площади для 1, 2 и 3 комнатных квартир
data_train.loc[data_train['Rooms'].isin([1,2,3]), ['Rooms', 'Square']].groupby(by='Rooms').median()

,Square
Rooms,
1.0,40.406590
2.0,55.841812
3.0,77.413643


In [28]:
# заменим значения для квартир с аномально большим количеством комнат на медианные
data_train.loc[data_train['Id'].isin([5927, 14003, 14865]), 'Rooms'] = 2

data_train.loc[data_train['Id'] == 8491, 'Rooms'] = 1

### Test train split

In [153]:
from sklearn.model_selection import train_test_split

# перед дальнейшей обработкой данных необходимо разбить датасет на train и valid
train, valid = train_test_split(data_train, test_size=0.3, random_state=RANDOM_STATE)

# проверяем размеры получившихся DataFrame
train.shape, valid.shape

((7000, 20), (3000, 20))

### Data prepare (подготовка данных)

В данном разделе расчитаем необходимые статистики. Дополним ими данные. Напишем функции для очистки данных основываясь на выводах из предидущего раздела. Обработаем категориальные признаки.

#### Категориальные признаки

In [127]:
# начнем с обработки категориальных признаков
# посмотрим как они выглядят
train.describe(include=['object'])

,Ecology_2,Ecology_3,Shops_2
count,7000,7000,7000
unique,2,2,2
top,B,B,B
freq,6936,6798,6416


In [128]:
# посмотрим какие встречаются значения
for cat_f in train.select_dtypes(include=['object']).columns:
    print(train[cat_f].unique())

['B' 'A']
['B' 'A']
['B' 'A']


In [129]:
def prepare_category(df, cat_feats=['Ecology_2', 'Ecology_3', 'Shops_2']):
    ''' Приводит категориальные признаки к числовым
    '''
    df_copy = df.copy()
    for f in cat_feats:
        df_copy.loc[:, f] = df[f].map({'A': 1, 'B': 0})
    return df_copy

In [154]:
train = prepare_category(train)
valid = prepare_category(valid)

#### Расчет статистик

In [155]:
# посчитаем различные статистики по train

# количество квартир в районе:
stats_distr_1 = train['DistrictId'].value_counts(normalize=True).reset_index().\
                                                                 rename(columns={'index':'DistrictId', 
                                                                                 'DistrictId':'flat_qty_distr'})

# средняя цена за квартиру по районам и кол-ву комнат
stats_distr_rooms_1 = train.groupby(['DistrictId', 'Rooms'], as_index=False)[['Price']].mean().\
                                                                            rename(columns={'Price':'mean_price_distr_rooms'})

# средняя цена за квартиру по кол-ву комнат
stats_rooms_1 = train.groupby(['Rooms'], as_index=False)[['Price']].mean().rename(columns={'Price':'mean_price_rooms'})

# медианная площадь по району и кол-ву комнат
stats_distr_rooms_2 = train.groupby(['DistrictId', 'Rooms'], as_index=False)[['Square']].median().\
                                                                            rename(columns={'Square':'mean_square_distr_rooms'})

# медианная площадь по кол-ву комнат
stats_rooms_2 = train.groupby(['Rooms'], as_index=False)[['Square']].median().\
                                                                            rename(columns={'Square':'mean_square_rooms'})

In [132]:
def add_stat_distr_1(df, stats=stats_distr_1):
    df_copy = df.copy()
    df_copy = pd.merge(df_copy, stats, on='DistrictId', how='left')
    df_copy['flat_qty_distr'] = df_copy['flat_qty_distr'].fillna(stats['flat_qty_distr'].min())
    return df_copy

In [133]:
def add_stat_distr_rooms(df, stats_1=stats_distr_rooms_1, stats_2=stats_rooms_1):
    df_copy = df.copy()
    df_copy = pd.merge(df_copy, stats_1, on=['DistrictId', 'Rooms'], how='left')
    df_copy = pd.merge(df_copy, stats_2, on='Rooms', how='left')
    
    df_copy['mean_price_rooms'] = df_copy['mean_price_rooms'].fillna(stats_2['mean_price_rooms'].mean())
    df_copy['mean_price_distr_rooms'] = df_copy['mean_price_distr_rooms'].fillna(df_copy['mean_price_rooms'])
    return df_copy

In [134]:
def add_stat_distr_rooms_2(df, stats_1=stats_distr_rooms_2, stats_2=stats_rooms_2):
    df_copy = df.copy()
    df_copy = pd.merge(df_copy, stats_1, on=['DistrictId', 'Rooms'], how='left')
    df_copy = pd.merge(df_copy, stats_2, on='Rooms', how='left')
    
    df_copy['mean_square_rooms'] = df_copy['mean_square_rooms'].fillna(stats_2['mean_square_rooms'].mean())
    df_copy['mean_square_distr_rooms'] = df_copy['mean_square_distr_rooms'].fillna(df_copy['mean_square_rooms'])
    return df_copy

In [156]:
train = add_stat_distr_1(train)
valid = add_stat_distr_1(valid)

train = add_stat_distr_rooms(train)
valid = add_stat_distr_rooms(valid)

train = add_stat_distr_rooms_2(train)
valid = add_stat_distr_rooms_2(valid)

#### Коррекция HouseFloor и Floor

In [136]:
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

In [137]:
# посмотрим сколькоэтажные дома встречаются чаще всего
train['HouseFloor'].value_counts().head(5)

17.0    1634
9.0      977
12.0     749
5.0      707
14.0     446
Name: HouseFloor, dtype: int64

In [138]:
import numpy as np

In [139]:
def coorect_HouseFloor(floor, sorted_HouseFloor):
    for housefloor in sorted_HouseFloor:
        if floor.item() <= housefloor:
            return housefloor
    return floor.item()  # если квартира на этаже который выше часто встречаемых HouseFloor


def prepare_HouseFloor(df, source_df):
    df_copy = df.copy()
    sorted_HouseFloor = list(source_df['HouseFloor'].value_counts().index)[:4]
    sorted_HouseFloor.sort()
    df_copy.loc[df_copy['HouseFloor'] == 0, 'HouseFloor'] = \
        df.loc[df['HouseFloor'] == 0, ['Floor']].apply(coorect_HouseFloor, axis=1, args=[sorted_HouseFloor])
    return df_copy


# решим проблему Floor > HouseFloor 
# для тех домов, где HouseFloor = 1 будем Floor менять местами с HouseFloor
def prepare_Floor_HouseFloor_ex(df):
    df_copy = df.copy()
    df_copy.loc[(df_copy['Floor'] > df_copy['HouseFloor']) & (df_copy['HouseFloor'] == 1), ['Floor', 'HouseFloor']] = \
        df.loc[(df['Floor'] > df['HouseFloor']) & (df['HouseFloor'] == 1), ['HouseFloor', 'Floor']].\
            rename(columns={'HouseFloor':'Floor', 'Floor':'HouseFloor'})
    return df_copy


# для тех домов, где HouseFloor > 1 будем Floor приравнивать к HouseFloor
def prepare_Floor_HouseFloor(df):
    df_copy = df.copy()
    df_copy.loc[(train['Floor'] > train['HouseFloor']) & (train['HouseFloor'] > 1), 'Floor'] = \
        df.loc[(train['Floor'] > train['HouseFloor']) & (train['HouseFloor'] > 1), 'HouseFloor']
    return df_copy



# создадим два признака первый этаж и последний этаж
def create_feat_last_floor(df):
    df_copy = df.copy()
    df_copy['LastFloor'] = (df['Floor'] == df['HouseFloor'])
    return df_copy


# создадим два признака первый этаж и последний этаж
def create_feat_first_floor(df):
    df_copy = df.copy()
    df_copy['FirstFloor'] = (df['Floor'] == 1)
    return df_copy

In [157]:
train = prepare_HouseFloor(train, source_df=train)
train = prepare_Floor_HouseFloor(train)
train = prepare_Floor_HouseFloor_ex(train)
train = create_feat_last_floor(train)
train = create_feat_first_floor(train)

valid = prepare_HouseFloor(valid, source_df=train)
valid = prepare_Floor_HouseFloor(valid)
valid = prepare_Floor_HouseFloor_ex(valid)
valid = create_feat_last_floor(valid)
valid = create_feat_first_floor(valid)


#### Коррекция KitchenSquare

In [141]:
# Если площадь кухни составляет > 80% от общей площади, то зануляем ее
def prepare_KitchenSquare(df):
    df_copy = df.copy()
    df_copy.loc[df_copy['KitchenSquare'] / df_copy['Square'] > 0.8, 'KitchenSquare'] = 0
    return df_copy

In [158]:
train = prepare_KitchenSquare(train)

valid = prepare_KitchenSquare(valid)

#### Коррекция LifeSquare и Square

In [147]:
# уберем все аномально большие занчения LifeSquare
def prepare_LifeSquare_max(df):
    df_copy = df.copy()
    df_copy.loc[(df_copy['LifeSquare'] / df_copy['Square']) > 1.2, 'LifeSquare'] = np.NaN
    return df_copy

In [123]:
# Поменяем местами LifeSquare и Square для тех квартир где LifeSquare > Square
def prepare_Square_LifeSquare_ex(df):
    df_copy = df.copy()
    df_copy.loc[(df_copy['LifeSquare'] > df_copy['Square']), ['LifeSquare', 'Square']] = \
        df.loc[(df['LifeSquare'] > df['Square']), ['Square', 'LifeSquare']].\
            rename(columns={'LifeSquare':'Square', 'Square':'LifeSquare'})
    return df_copy

In [161]:
train = prepare_LifeSquare_max(train)
valid = prepare_LifeSquare_max(valid)

train = prepare_Square_LifeSquare_ex(train)
valid = prepare_Square_LifeSquare_ex(valid)

In [163]:
# посмотрим на отношение LifeSquare к Square
(train['LifeSquare'] / train['Square']).median()

0.6170119590685053

In [164]:
# считаем статистику по соотношению LifeSquare к Square
stats_LifeSquare_Square_perc = train.loc[train['LifeSquare'] > 0, ['Rooms', 'Square', 'LifeSquare']]\
    .groupby(by=['Rooms'], as_index=False).median()

stats_LifeSquare_Square_perc['LS_S_perc'] = \
    stats_LifeSquare_Square_perc['LifeSquare'] / stats_LifeSquare_Square_perc['Square']

stats_LifeSquare_Square_perc = stats_LifeSquare_Square_perc.drop(['Square', 'LifeSquare'], axis=1)

In [89]:
# добавляем статистику в датасет
def add_stat_LifeSquare_Square_perc(df, stats=stats_LifeSquare_Square_perc):
    df_copy = df.copy()
    df_copy = pd.merge(df_copy, stats, on='Rooms', how='left')
    df_copy['LS_S_perc'] = df_copy['LS_S_perc'].fillna(stats['LS_S_perc'].median())
    return df_copy

# заполняем поле LifeSquare
def fillna_LifeSquare(df):
    df_copy = df.copy()
    df_copy.loc[df_copy['LifeSquare'].isna(), 'LifeSquare'] = df_copy.loc[df_copy['LifeSquare'].isna(), 'Square'] * \
                                                                df_copy.loc[df_copy['LifeSquare'].isna(), 'LS_S_perc']
    return df_copy

# так же для всех квартиры у которых соотношение LifeSquare к Square < 30% заменим LifeSquare
def prepare_LifeSquare_small(df):
    df_copy = df.copy()
    df_copy.loc[(df_copy['LifeSquare'] / df_copy['Square']) < 30, 'LifeSquare'] = \
        df_copy.loc[(df_copy['LifeSquare'] / df_copy['Square']) < 30, 'Square'] * \
            df_copy.loc[(df_copy['LifeSquare'] / df_copy['Square']) < 30, 'LS_S_perc']
    return df_copy

In [165]:
train = add_stat_LifeSquare_Square_perc(train)
train = fillna_LifeSquare(train)
train = prepare_LifeSquare_small(train)

valid = add_stat_LifeSquare_Square_perc(valid)
valid = fillna_LifeSquare(valid)
valid = prepare_LifeSquare_small(valid)

In [93]:
def prepare_Square(df):
    df_copy = df.copy()
    df_copy.loc[df_copy['Square'] < 15, 'Square'] = df.loc[df['Square'] < 15, 'mean_square_distr_rooms']
    return df_copy

In [167]:
train = prepare_Square(train)

valid = prepare_Square(valid)

In [172]:
# заменим nan в поле Healthcare_1 на 0
def fillna_Healthcare_1(df):
    df_copy = df.copy()
    df_copy['Healthcare_1'] = df_copy['Healthcare_1'].fillna(0)
    return df_copy

In [175]:
train = fillna_Healthcare_1(train)
valid = fillna_Healthcare_1(valid)

### Model

In [179]:
from sklearn.ensemble import RandomForestRegressor as RF
from sklearn.metrics import r2_score as r2, mean_squared_error as mse

In [188]:
def fit_and_score(model, X_train, X_valid, y_train, y_valid):
    model.fit(X_train, y_train.values.ravel())
    y_train_pred = model.predict(X_train)
    y_valid_pred = model.predict(X_valid)
    r2_train = r2(y_train, y_train_pred)
    r2_valid = r2(y_valid, y_valid_pred)
    return r2_train, r2_valid

In [228]:
rf = RF(random_state=RANDOM_STATE,
        n_estimators=1000,
        max_depth=14,
        max_features=16,
        n_jobs=-1)

In [231]:
feats = [
#     'DistrictId',
    'Rooms',
    'Square',
    'LifeSquare',
    'KitchenSquare',
    'Floor',
    'HouseFloor',
    'HouseYear',
    'Ecology_1',
    'Ecology_2',
    'Ecology_3',
    'Social_1',
    'Social_2', 
    'Social_3', 
    'Healthcare_1',
    'Helthcare_2', 
    'Shops_1', 
    'Shops_2', 
    'flat_qty_distr',
    'mean_price_distr_rooms', 
    'mean_price_rooms', 
    'mean_square_distr_rooms',
    'mean_square_rooms',
    'LastFloor', 
    'FirstFloor', 
#     'LS_S_perc'
]

target = ['Price']

In [232]:
fit_and_score(rf, train[feats], valid[feats], train[target], valid[target])

(0.9513131543759897, 0.7029320252194721)